# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import citipy
# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#Here I stablish a path to the cities.csv file
herepath = os.path.dirname(os.path.abspath('__file__'))
weather_data = os.path.join(herepath,'cities.csv')

#Then we transformed it into a Pandas Data Frame
Cities_weather = pd.read_csv(weather_data)
Cities_weather = Cities_weather.drop(['Unnamed: 0'], axis=1)
Cities_weather.head()

,City,Country,Longitud,Latitud,Date,Max. Temperature,Humidity,Wind Speed,Cloudiness
0,Saint Anthony,US,-93.22,45.02,1586371289,12.78,34,6.70,90
1,Bredasdorp,ZA,20.04,-34.53,1586371290,16.00,72,6.20,99
2,Porbandar,IN,69.61,21.64,1586371290,25.30,85,6.00,0
3,Troitskoye,RU,84.68,52.98,1586371290,1.94,89,1.71,100
4,Cape Town,ZA,18.42,-33.93,1586370996,17.78,72,4.60,96


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [32]:
#Here we define the asked parameters as variables
locations = Cities_weather[["Latitud", "Longitud"]]
humidity = Cities_weather['Humidity']
max_int = Cities_weather['Humidity'].max()

#Then we create a gmaps figure
fig = gmaps.figure()
#And here we use the tool 'heatmap_layer' from gmaps to create a heatmap layer
humidity_heat_map = gmaps.heatmap_layer(locations, weights = humidity, max_intensity = max_int, point_radius = 20)
#And then we add this layer to the previos figure we create
fig.add_layer(humidity_heat_map)
#We show the figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
#Here we uso loc function to select the data we desire, with cloudiness less than ten, wind speed less than 5 and Temperature between 21 to 25
Ideal_weather = Cities_weather.loc[((Cities_weather['Cloudiness'] < 10) & (Cities_weather['Wind Speed'] < 5) & (Cities_weather['Max. Temperature'].between(21,25)))]

#Ideal_weather.count()    
#There's no null values :)
Ideal_weather
                                

,City,Country,Longitud,Latitud,Date,Max. Temperature,Humidity,Wind Speed,Cloudiness
39,Carnarvon,AU,113.63,-24.87,1586371300,24.00,83,3.10,7
87,Manakara,MG,48.02,-22.13,1586371311,24.27,89,3.26,4
95,Faya,SA,42.45,18.39,1586371314,21.00,52,2.10,0
109,Kapit,MY,112.93,2.02,1586371317,23.09,94,0.21,3
121,Ancud,CL,-73.82,-41.87,1586371320,23.00,35,2.10,0
179,Maţāy,EG,30.78,28.42,1586371335,21.18,31,3.89,0
198,Coyhaique,CL,-72.07,-45.58,1586371340,21.00,52,1.50,0
215,Pisco,PE,-76.22,-13.70,1586371345,23.89,30,3.10,8
277,Ţahţā,EG,31.50,26.77,1586371361,23.00,25,4.10,0
388,Saint George,US,-113.58,37.10,1586371370,22.22,18,2.10,1


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
#Store into variable named hotel_df.
hotel_df = pd.DataFrame(Ideal_weather)
#Add a "Hotel Name" column to the DataFrame.
hotel_df['Hotel Name'] = ''
hotel_df.head()

,City,Country,Longitud,Latitud,Date,Max. Temperature,Humidity,Wind Speed,Cloudiness,Hotel Name
39,Carnarvon,AU,113.63,-24.87,1586371300,24.00,83,3.10,7,
87,Manakara,MG,48.02,-22.13,1586371311,24.27,89,3.26,4,
95,Faya,SA,42.45,18.39,1586371314,21.00,52,2.10,0,
109,Kapit,MY,112.93,2.02,1586371317,23.09,94,0.21,3,
121,Ancud,CL,-73.82,-41.87,1586371320,23.00,35,2.10,0,


In [13]:
#Set parameters to search for hotels with 5000 meters.
url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?'
area = 5000
kind = 'hotel'
key = g_key

#We create a params dictionary in order to respond the api parameters that google asks
#The only parameter who is missed is location, but we will add further while we iterate in the df
params = { 
    'radius': area,
    'type' : kind,
    'key': key}

hotel_df['Hotel Location'] = ""

#Hit the Google Places API for each city's coordinates.
#First we iterate trhoug different cordenaste to create the differente api calls
for index, row in hotel_df.iterrows():
    lat = row['Latitud']
    lon = row['Longitud']
    city = row['City']
    country = row['Country']
    #Here we sort cordenates as google documentation ask 
    cordenates = f'{lat},{lon}'
    #And add them to the params dictionary we create
    params['location'] = cordenates
    #Create a response variable to store our request
    response = requests.get(url, params=params).json()
    #As google documentations says, the data we look for are into a 'results' list in a dictionary so we grab just that
    results = response['results']
    
    #If there is a registered hotel in the city we take the data of the closest and add it to our hotel_df
    try:
        print(f"Closest hotel to {city},{country} is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Aparently there's no hotels here... skipping.")
        pass
    print("------------")

Closest hotel to Carnarvon,AU is Carnarvon.
------------
Closest hotel to Manakara,MG is Manakara.
------------
Closest hotel to Faya,SA is الغال.
------------
Closest hotel to Kapit,MY is Kapit.
------------
Closest hotel to Ancud,CL is Ancud.
------------
Closest hotel to Maţāy,EG is Matay.
------------
Closest hotel to Coyhaique,CL is Coyhaique.
------------
Closest hotel to Pisco,PE is Pisco.
------------
Closest hotel to Ţahţā,EG is Tahta.
------------
Closest hotel to Saint George,US is St. George.
------------
Closest hotel to Werda,BW is Werda.
------------
Closest hotel to Griffith,US is Gary.
------------
Closest hotel to Concordia,AR is Salto.
------------
Closest hotel to Tura,IN is Tura.
------------
Closest hotel to Marzuq,YE is Al Hujayrah.
------------
Closest hotel to Uíge,AO is Uige.
------------
Closest hotel to Kahului,US is Kahului.
------------
Closest hotel to Bowen,AU is Bowen.
------------
Closest hotel to San Vicente,AR is San Pedro.
------------
Closest hotel

In [33]:
#Here is just get rid of some columns I created while experimenting
hotel_df = hotel_df.drop('Hotel Location', axis=1)
hotel_df = hotel_df.drop('location', axis=1)
hotel_df

KeyError: "['Hotel Location'] not found in axis"

In [37]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Hotel</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations2 = hotel_df[["Latitud", "Longitud"]]

In [38]:
marker = gmaps.marker_layer(locations2, info_box_content= hotel_info)
# Add marker layer ontop of heat map
fig.add_layer(marker)
# Display Map
fig

Figure(layout=FigureLayout(height='420px'))